Arabidopsis seedlings are incubated with certain pharmaceuticals that attack the cell wall of the plants. This leads to increased production of lignin, a polymer that causes the seedlings to "stiffen" so that nothing can get through the cell wall anymore. For analysis, we use a specific dye that docks onto the individual lignin polymers, making them visible. And this is exactly where our interest lies. Using Fiji or ImageJ, we have to manually quantify the stained regions for every sample we analyse, sometimes several hundreds in number. Instead, we would like to have a tool that can be fed with all the images and quantifies the stained regions within seconds.

<img src="images/root0001.tif" width="400" height="200">

In [7]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import napari
import sys
import os

In [ ]:
# Classes and functions

In [4]:
# assign directory
directory = 'images/root_images'
 
# iterate over files in directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)

images/root_images\root0001.tif
images/root_images\root0002.tif
images/root_images\root0003.tif
images/root_images\root0004.tif
images/root_images\root0005.tif
images/root_images\root0006.tif
images/root_images\root0007.tif
images/root_images\root0008.tif
images/root_images\root0009.tif
images/root_images\root0010.tif


In [9]:
# Read image
img = cv.imread("images/root_images/root0004.tif")
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# Visualize images using napari (will open in a separate window)
# Create an empty viewer
viewer = napari.Viewer()
# Add images as layers
viewer.add_image(img, rgb=True, name='Original Image')
viewer.add_image(img_gray, name='Grayscale Image')

<Image layer 'Grayscale Image' at 0x29fc80351b0>

In [10]:
[x, y] = img_gray.shape
max_row = np.zeros(x)
min_row = np.zeros(x)
boolean = np.full(x, False)

for i in range(x):
    max_row[i] = np.max(img_gray[i,:])
    min_row[i] = np.min(img_gray[i,:])
    boolean[i] = max_row[i]-min_row[i]>=50

In [11]:
boolean_cp = boolean.copy()

boolean_cp[:99] = False
boolean_cp[-99:] = False

for i in range(100,x-100):
    if not(boolean_cp[i] == True and sum(boolean_cp[i-10:i+10])>=10):
        boolean_cp[i] = False

idx_upper = np.where(boolean_cp == True)[0][0] - 50
idx_lower = np.where(boolean_cp == True)[0][-1] + 50

In [12]:
# Crop region of interest
img_gray_cp = img_gray.copy()
# img_gray_cp[:idx_upper,:] = 255
# img_gray_cp[idx_lower:,:] = 255

img_gray_cp[:225] = 255
img_gray_cp[-225:] = 255

viewer.add_image(img_gray_cp, name='Cropped Image')

<Image layer 'Cropped Image' at 0x29fc881ba60>

In [115]:
# Global thresholding
_, thresh1 = cv.threshold(img_gray_cp,100,255,cv.THRESH_BINARY)
_, thresh2 = cv.threshold(img_gray_cp,125,255,cv.THRESH_BINARY)
_, thresh3 = cv.threshold(img_gray_cp,150,255,cv.THRESH_BINARY)

titles = ['Global Thresh @100','Global Thresh @125','Global Thresh @150']
images = [thresh1, thresh2, thresh3]

for i in range(len(images)):
    viewer.add_image(images[i], name=titles[i])

In [13]:
min = np.min(img_gray_cp[idx_upper+1:idx_lower-1,:])
max = np.max(img_gray_cp[idx_upper+1:idx_lower-1,:])
avg = np.average(img_gray_cp[idx_upper+1:idx_lower-1,:])
print(min, max, avg)

q_50 = np.quantile(img_gray_cp[idx_upper+1:idx_lower-1,:],0.50)
q_10 = np.quantile(img_gray_cp[idx_upper+1:idx_lower-1,:],0.10)
q_01 = np.quantile(img_gray_cp[idx_upper+1:idx_lower-1,:],0.01)
print(q_01, q_10, q_50)

print(np.average(img_gray_cp[img_gray_cp <= q_01]),np.average(img_gray_cp[img_gray_cp <= q_10]),np.average(img_gray_cp[img_gray_cp <= q_50]))

55 202 169.93820121593046
108.0 147.0 176.0
94.60598321780373 131.47368790838192 159.88508091935265


In [14]:
th = np.average(img_gray_cp[img_gray_cp <= q_10]) - 2
_, threshc = cv.threshold(img_gray_cp,th,255,cv.THRESH_BINARY)

kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(4,4))
op_threshc = cv.morphologyEx(threshc, cv.MORPH_OPEN, kernel)

viewer.add_image(threshc, name='10th-quantile thresh')
viewer.add_image(op_threshc, name='opened 10th-quantile thresh')

<Image layer 'opened 10th-quantile thresh' at 0x29fa73f8ee0>

In [15]:
copy = op_threshc.copy()
copy[copy == 0] = True
copy[copy == 255] = False
copy = copy.astype('bool')

print("Grayscale intensity normalized over (what is assumed to be the) stained region: " + str(np.sum(img_gray[copy])/np.sum(copy)))
print("Grayscale intensity normalized over cropped region: " + str(np.sum(img_gray[copy])/((idx_lower-idx_upper)*y)))
print("Grayscale intensity normalized over whole image: " + str(np.sum(img_gray[copy])/(x*y)))

Grayscale intensity normalized over (what is assumed to be the) stained region: 115.10473198133866
Grayscale intensity normalized over cropped region: 4.4052625058302235
Grayscale intensity normalized over whole image: 1.5372530619303386


In [122]:
cv.imwrite("images/0004/grayscale.tif", img_gray)
cv.imwrite("images/0004/cropped_grayscale.tif", img_gray_cp)
cv.imwrite("images/0004/masking.tif", threshc)
cv.imwrite("images/0004/cleaned_masking.tif", op_threshc)

True

In [ ]:
# var  = img.astype('uint16')

# [x, y, z] = var.shape 
# for i in range(x):
#     sum_min = np.min(var[i,:,0]+var[i,:,1]+var[i,:,2])
#     sum_max = np.max(var[i,:,0]+var[i,:,1]+var[i,:,2])
#     diff_sum = sum_max - sum_min
#     print(np.max(var[i,:,0])-np.min(var[i,:,0]), np.max(var[i,:,1])-np.min(var[i,:,1]), np.max(var[i,:,2])-np.min(var[i,:,2]), sum_min, sum_max, diff_sum)

In [8]:
# # Adapative thresholding
# thm = cv.adaptiveThreshold(img_gray_cp,255,cv.ADAPTIVE_THRESH_MEAN_C,cv.THRESH_BINARY,31,2)
# thg = cv.adaptiveThreshold(img_gray_cp,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,cv.THRESH_BINARY,31,2)

# kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE,(4,4))
# closing = cv.morphologyEx(thm, cv.MORPH_CLOSE, kernel)

# closing = cv.morphologyEx(thresh2, cv.MORPH_CLOSE, kernel)
# num,labels,stats,centroids = cv.connectedComponentsWithStats(closing,connectivity=8)
# for i in range(num):
#     print(np.sum(labels==i),stats[i][4])

<Image layer 'closing' at 0x28b8c87df60>